# Imports and pickle

In [6]:
import pandas as pd
import pickle
import re
import os
import PyPDF2
from joblib import Parallel, delayed

In [7]:
# Define the folder where your .pkl files are located
folder_path = "/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/pickel_22_copy"

# Initialize an empty dictionary to store loaded DataFrames
loaded_dfs = {}

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".pkl"):
        file_path = os.path.join(folder_path, filename)
        
        # Load the pickled object
        with open(file_path, 'rb') as f:
            loaded_object = pickle.load(f)
        
        # Check if the loaded object is a DataFrame
        if isinstance(loaded_object, pd.DataFrame):
            loaded_dfs[filename] = loaded_object
        elif isinstance(loaded_object, list):
            # Check if the list contains DataFrames
            if all(isinstance(item, pd.DataFrame) for item in loaded_object):
                # Convert list of DataFrames to a single DataFrame
                concatenated_df = pd.concat(loaded_object)
                loaded_dfs[filename] = concatenated_df
            else:
                print(f"Skipping {filename}, list does not contain DataFrames.")
        else:
            print(f"Skipping {filename}, not a DataFrame or list of DataFrames.")

# Concatenate individual DataFrames into one big DataFrame
if loaded_dfs:
    df = pd.concat(loaded_dfs.values(), keys=loaded_dfs.keys())
else:
    print("No DataFrames were loaded.")

# Missing values

In [8]:
df.shape

(5120245, 8)

In [4]:
df.isna().sum()

Wahlperiode          0
Sitzung              0
Date                 0
Start           865122
Schluss         921065
Speaker              0
Text_Spoken          0
Reactions      4324247
dtype: int64

# Rename columns

In [10]:
# Renaming the column
df.rename(columns={'Wahlperiode': 'Election_Period', 'Sitzung': 'Session', 'Schluss': 'End_Time'}, inplace=True)

# Ending-Time

## Find Schluss Pattern

In [11]:
df.head()

Election_Period Session              Date  Start End_Time  \
dfs_batch_58.pkl 1              20     004  7. Dezember 2021  12:00    13:41   
                 2              20     004  7. Dezember 2021  12:00    13:41   
                 3              20     004  7. Dezember 2021  12:00    13:41   
                 4              20     004  7. Dezember 2021  12:00    13:41   
                 5              20     004  7. Dezember 2021  12:00    13:41   

                                   Speaker  \
dfs_batch_58.pkl 1  Präsidentin Bärbel Bas   
                 2  Präsidentin Bärbel Bas   
                 3  Präsidentin Bärbel Bas   
                 4  Präsidentin Bärbel Bas   
                 5  Präsidentin Bärbel Bas   

                                                          Text_Spoken  \
dfs_batch_58.pkl 1  Ich wünsche Ihnen allen einen schönen guten Ta...   
                 2                          Die Sitzung ist eröffnet.   
                 3  Vor Eintritt in die Tagesordnung: Liebe Kolleg...   
                 4  Liebe Kolleginnen und Kollegen, die Covid 19 P...   
                 5  Laut Robert Koch Institut sind die aktuellen F...   

                   Reactions  
dfs_batch_58.pkl 1      None  
                 2      None  
                 3      None  
                 4      None  
                 5      None

In [12]:
# Identify missing Schluss
missing_schluss_df = df[df['End_Time'].isna()]
missing_schluss_tuples = missing_schluss_df.groupby(['Election_Period', 'Session']).size().reset_index(name='Missing_Count')

In [13]:
missing_schluss_tuples

,Election_Period,Session,Missing_Count
0,15,042,1660
1,15,083,2665
2,16,075,2216
3,16,130,4387
4,16,186,6539
...,...,...,...
184,19,107,11118
185,19,108,4939
186,19,109,694
187,19,112,4663


## Find a way to replace multiple at the same time

In [14]:
# Let us get the text to look for other Schluss patterns:
def extract_text_from_pdf(pdf):
    """

    :param pdf: 
    :return: 
    """
    with open(pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text

In [15]:
def extract_schluss(path):
    pdf_text = extract_text_from_pdf(path)
    schluss_time = None

    # Search for the pattern "Die Sitzung ist geschlossen. (Schluss: <time>)"
    pattern = r"\Schluss\:*\s*(\d{2}\s*\d*)"
    match = re.search(pattern, pdf_text)

    if match:
        # Extract the captured group which contains the time
        schluss_time = match.group(1).strip()

    return schluss_time

In [17]:
def extract_schluss_from_row(row):
    wahlperiode = row['Election_Period']
    sitzung = row['Session']
    
    pdf_path = f'/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/1998_2023/{wahlperiode}{sitzung}.pdf'
    schluss_time = extract_schluss(pdf_path)
    
    return (wahlperiode, sitzung, schluss_time)

results = Parallel(n_jobs=-2)(delayed(extract_schluss_from_row)(row) for _, row in missing_schluss_tuples.iterrows())

# Initialize a dictionary to hold new Schluss times
new_schluss = {}

for wahlperiode, sitzung, schluss_time in results:
    if schluss_time:
        new_schluss[(wahlperiode, sitzung)] = schluss_time

In [ ]:
# Fill in the new Schluss times into the DataFrame
for (wahlperiode, sitzung), schluss_time in new_schluss.items():
    df.loc[(df['Election_Period'] == wahlperiode) & (df['Session'] == sitzung), 'End_Time'] = schluss_time

In [ ]:
# Check how many missing values left
# Identify missing Schluss
missing_schluss_df = df[df['End_Time'].isna()]
missing_schluss_tuples = missing_schluss_df.groupby(['Election_Period', 'Session']).size().reset_index(name='Missing_Count')
missing_schluss_tuples

In [ ]:
# 14 042
df.loc[(df['Election_Period'] == '15') & (df['Session'] == '083')].tail(20)

In [ ]:
df1 = df.copy()

## Drop all after (Schluss including Schluss)

In [ ]:
# Initialize an empty list to collect cases where the sentence is not found
not_found_list = []

def truncate_after_session_closed(group):
    # Pattern to match variations of "Die Sitzung ist geschlossen."
    pattern = re.compile(r"\(Schluss", re.IGNORECASE)
    
    # Find index where pattern matches
    idx_closed = group[group['Text_Spoken'].apply(lambda x: bool(pattern.search(str(x))))].index
    
    # If pattern not found, append to not_found_list
    if len(idx_closed) == 0:
        not_found_list.append({'Election_Period': group['Election_Period'].iloc[0], 'Session': group['Session'].iloc[0]})
    
    return group.loc[:idx_closed[-1]] if len(idx_closed) > 0 else group

# Group by 'Election_Period' and 'Session' and apply the function
df_1grouped = df1.groupby(['Election_Period', 'Session'])
df_1truncated = df_1grouped.apply(truncate_after_session_closed).reset_index(drop=True)

# Convert not_found_list to a DataFrame
not_found_df = pd.DataFrame(not_found_list)

print(not_found_df)

In [ ]:
df2 = df_1truncated.copy()

In [ ]:
df1.shape[0]-df2.shape[0]

In [ ]:
df2.isna().sum()

# Start

## Find Missing Start Time

In [ ]:
# Filter rows where the Start column is missing in df_1
missing_start_df = df2[df2['Start'].isna()]

# Group by Wahlperiode and Sitzung and count the number of missing starts for each group
grouped_missing_starts = missing_start_df.groupby(['Election_Period', 'Session']).size().reset_index(name='Missing_Count')

# Display the result
print(grouped_missing_starts)

In [ ]:
# Function to extract the start time
def extract_start(path):
    pdf_text = extract_text_from_pdf(path)
    start_time = None
    
    # Search for the word "Beginn:"
    start_idx = pdf_text.find("Beginn")
    
    if start_idx != -1:
        # Extract the following text (assuming time format as HH:MM, hence taking 5 characters)
        start_time = pdf_text[start_idx + len("Beginn"): start_idx + len("Beginn") + 5].strip()
        
    return start_time

In [ ]:
# Step 1: Identify missing rows
missing_start_tuples = grouped_missing_starts[['Election_Period', 'Session']].apply(tuple, axis=1)

# Initialize a dictionary to hold the new Start times
new_starts = {}

# Step 2: Loop through PDFs
for wahlperiode, sitzung in missing_start_tuples:
    pdf_path = f"/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Final_Project/speeches_bundestag/1998_2023/{wahlperiode}{sitzung}.pdf"  # Replace with your actual folder path
    
    try:
        # Extract Start time
        time = extract_start(pdf_path)
        
        if time:  # Check if time is not None or empty
            new_starts[(wahlperiode, sitzung)] = time
    except Exception as e:
        print(f"An error occurred while processing {pdf_path}. Error: {e}")

In [ ]:
new_starts

In [ ]:
len(new_starts)

In [ ]:
# Step 3: Update DataFrame
for (wahlperiode, sitzung), start_time in new_starts.items():
    df2.loc[(df2['Election_Period'] == wahlperiode) & (df2['Session'] == sitzung), 'Start'] = start_time

## Drop All Before Beginn

In [ ]:
def truncate_after_session_closed(group):
    # Pattern to match variations of "Die Sitzung ist geschlossen."
    pattern = re.compile(r"Beginn", re.IGNORECASE)
    
    # Find index where pattern matches
    idx_start = group[group['Text_Spoken'].apply(lambda x: bool(pattern.search(str(x))))].index
    
    # If pattern not found, append to not_found_list
    if len(idx_start) == 0:
        not_found_list.append({'Election_Period': group['Election_Period'].iloc[0], 'Session': group['Session'].iloc[0]})
    else:
        # Select the first index where the pattern is found and add one
        idx_start = idx_start[0] + 1
        # Return the truncated group starting from idx_start to the end
        return group.loc[idx_start:]
    
    # If pattern not found, return the group unmodified
    return group

# Group by 'Election_Period' and 'Session' and apply the function
df_2grouped = df2.groupby(['Election_Period', 'Session'])
df_2truncated = df_2grouped.apply(truncate_after_session_closed).reset_index(drop=True)

# Convert not_found_list to a DataFrame
not_found_df = pd.DataFrame(not_found_list)

print(not_found_df)

In [ ]:
df3 = df_2truncated.copy()

In [ ]:
df3.shape

In [ ]:
df3.isna().sum()

# Datatpyes

In [ ]:
df3.dtypes

## Wahlperiode & Sitzung

In [ ]:
df3['Election_Period'] = df3['Election_Period'].astype(int)
df3['Session'] = df3['Session'].astype(int)

In [ ]:
df4 = df3.copy()

## Date

In [ ]:
import calendar

In [ ]:
# Create a mapping of German month names to English
german_to_english = {
    'Januar': 'January',
    'Februar': 'February',
    'März': 'March',
    'April': 'April',
    'Mai': 'May',
    'Juni': 'June',
    'Juli': 'July',
    'August': 'August',
    'September': 'September',
    'Oktober': 'October',
    'November': 'November',
    'Dezember': 'December'
}

In [ ]:
# Replace German month names with English month names
df4['Date'] = df4['Date'].replace(german_to_english, regex=True)

# Now convert to datetime
df4['Temp_Date'] = pd.to_datetime(df4['Date'], format='%d. %B %Y', errors='coerce')

In [ ]:

# Create a boolean mask for entries where the conversion failed
mask = df4['Temp_Date'].isna()

# Group by the original 'Date' column and additional columns for rows where the conversion failed
problematic_dates = df4[mask].groupby(['Election_Period', 'Session', 'Date']).size().reset_index(name='Count')

# Sort by 'Count' to easily spot frequently occurring problematic entries
problematic_dates = problematic_dates.sort_values(by='Count', ascending=False)

# Drop the temporary 'Temp_Date' column
df4.drop(columns=['Temp_Date'], inplace=True)

# Print out the problematic dates along with 'Election_Period' and 'Session'
print("Counts of problematic entries by Wahlperiode, Sitzung, and Date:")
print(problematic_dates)

In [ ]:
# Manually update the Date here
df4.loc[(df4['Election_Period'] == 15) & (df4['Session'] == 16), 'Date'] = '19. December 2002'

In [ ]:
df4['Date'] = pd.to_datetime(df4['Date'], format='%d. %B %Y')

In [ ]:
df5 = df4.copy()

## Start

In [ ]:
# First inspect how Start can looks like I think something like this
# dd needs to be lower than 24
# dd dd
# letters

In [ ]:
# Strip
df5['Start'] = df5['Start'].str.strip()
df5['Start'] = df5['Start'].str.replace(r'\s+', '', regex=True)

In [ ]:
def clean_start_time(t):
    # Check for any non-digit characters
        # Remove all whitespaces from the input
    t = re.sub(r'\s+', '', t)
    
    # Remove all characters except digits and colon
    t = re.sub(r'[^\d:]', '', t)
    if re.search('[a-zA-Z]', t):
        return None
    
    elif re.match(r'^:\d{1}$', t):
        return f"0{t[1]}:00:00"
        # Handle cases like "13:45" or 9:33 :9
    
    elif re.match(r'^:\d{2}$', t):
        return f"{t[1]}:00:00"
    
    elif ":" in t:
        parts = t.split(':')
        if len(parts) == 2:
            # Check if first part is empty and second part is a single digit
            if parts[0] == '' and parts[1].isdigit() and len(parts[1]) == 1:
                return f"0{parts[1]}:00:00"
            elif parts[0] == '' and parts[1].isdigit() and len(parts[1]) == 2:
                return f"{parts[1]}:00:00"
            # Existing checks for parts with length of 2 each
            elif all(part.isdigit() for part in parts):
                hours, minutes = parts
                if int(hours) < 24 and int(minutes) < 60:
                    return f"{hours.zfill(2)}:{minutes.zfill(2)}:00"
    
    # Length of 4 indicates HHMM format
    if len(t) == 4:
        return f"{t[:2]}:{t[2:]}:00"
    
    # Length of 3 indicates HMM format
    elif len(t) == 3:
        # Check if minutes exceed 59
        if int(t[1:]) < 60:
            return f"0{t[0]}:{t[1:]}:00"
        elif int(t[:1]) < 24:
            return f"0{t[:1]}:{t[2]}:00"
    
    # Length of 2 could be HH format
    elif len(t) == 2:
        # Check if it's a valid hour
        if int(t) <= 24:
            return f"{t}:00:00"

    # Length of 1 indicates a single digit hour
    elif len(t) == 1:
        return f"{t}:00:00"

    # Handle cases like "13 45"
    elif " " in t:
        hours, minutes = t.split()
        if int(hours) <= 24 and int(minutes) < 60:
            return f"{int(hours)}:{int(minutes)}:00"
        
    else:
        # If none of the above conditions met, return None
        return None

In [ ]:
# Apply the function to the Start column
df5['Start'] = df5['Start'].apply(lambda x: clean_start_time(str(x)))

18 138

In [ ]:
# Now convert the cleaned times to timedelta
df5['Start'] = pd.to_timedelta(df5['Start'].dropna())

In [ ]:
df5.sample(10)

In [ ]:
df6 = df5.copy()

## Schluss

In [ ]:
def clean_schluss_time(t):    # Check for any non-digit characters
        # Remove all whitespaces from the input
    t = re.sub(r'\s+', '', t)
    
    # Remove all characters except digits and colon
    t = re.sub(r'[^\d:]', '', t)
    if re.search('[a-zA-Z]', t):
        return None
    
    elif re.match(r'^:\d{1}$', t):
        return f"0{t[1]}:00:00"
        # Handle cases like "13:45" or 9:33 :9
    
    elif re.match(r'^:\d{2}$', t):
        return f"{t[1]}:00:00"
    
    elif ":" in t:
        parts = t.split(':')
        if len(parts) == 2:
            # Check if first part is empty and second part is a single digit
            if parts[0] == '' and parts[1].isdigit() and len(parts[1]) == 1:
                return f"0{parts[1]}:00:00"
            elif parts[0] == '' and parts[1].isdigit() and len(parts[1]) == 2:
                return f"{parts[1]}:00:00"
            # Existing checks for parts with length of 2 each
            elif all(part.isdigit() for part in parts):
                hours, minutes = parts
                if int(hours) < 24 and int(minutes) < 60:
                    return f"{hours.zfill(2)}:{minutes.zfill(2)}:00"
    
    # Length of 4 indicates HHMM format
    if len(t) == 4:
        return f"{t[:2]}:{t[2:]}:00"
    
    # Length of 3 indicates HMM format
    elif len(t) == 3:
        # Check if minutes exceed 59
        if int(t[1:]) < 60:
            return f"0{t[0]}:{t[1:]}:00"
        elif int(t[:1]) < 24:
            return f"0{t[:1]}:{t[2]}:00"
    
    # Length of 2 could be HH format
    elif len(t) == 2:
        # Check if it's a valid hour
        if int(t) <= 24:
            return f"{t}:00:00"

    # Length of 1 indicates a single digit hour
    elif len(t) == 1:
        return f"{t}:00:00"

    # Handle cases like "13 45"
    elif " " in t:
        hours, minutes = t.split()
        if int(hours) <= 24 and int(minutes) < 60:
            return f"{int(hours)}:{int(minutes)}:00"
        
    else:
        # If none of the above conditions met, return None
        return None

In [ ]:
# Apply the function to the Start column
df6['End_Time'] = df6['End_Time'].apply(lambda x: clean_schluss_time(str(x)))

In [ ]:
# Now convert the cleaned times to timedelta
df6['End_Time'] = pd.to_timedelta(df6['End_Time'].dropna())

In [ ]:
df6.head()

In [ ]:
df7 = df6.copy()

# Impute Missing Values

In [ ]:
df7.isna().sum()

## Start

In [ ]:
# Filter rows where the Start column is missing in df_1
missing_start_df = df7[df7['Start'].isna()]

# Group by Wahlperiode and Sitzung and count the number of missing starts for each group
grouped_missing_starts = missing_start_df.groupby(['Election_Period', 'Session']).size().reset_index(name='Missing_Count')

# Display the result
print(grouped_missing_starts)

In [ ]:
# Let me do Start by Hand
manual_starts = {
    (18, 142): '12:30:00',
    (19, 6): '13:00:00',
    (19, 70): '13:00:00',
    (19, 82): '13:00:00',
}

In [ ]:
# Iterate over the manual_starts items
for (wahlperiode, sitzung), start_time in manual_starts.items():
    # Convert start_time to a timedelta object
    start_time_delta = pd.to_timedelta(start_time)
    print(start_time_delta)
    # Apply the timedelta where conditions are met
    df7.loc[(df7['Election_Period'] == wahlperiode) & (df7['Session'] == sitzung), 'Start'] = start_time_delta

In [ ]:
df7.isna().sum()

In [ ]:
df8 = df7.copy()

## Schluss

In [ ]:
df8.isna().sum()

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
# Calculate the duration only for rows where 'End_Time' is not None
df8['Dauer'] = df8.apply(lambda row: row['End_Time'] - row['Start'] if pd.notna(row['End_Time']) else None, axis=1)

# Now convert 'Dauer' to seconds only for non-None values
df8['Dauer_in_Sekunden'] = df8['Dauer'].dt.total_seconds()

# Prepare your features and target variable, excluding rows where 'Dauer' is None
X = df8[['Election_Period', 'Session']]
y = df8['Dauer_in_Sekunden']

# Use only the rows with non-missing 'Dauer_in_Sekunden' to train the model
X_train = X[y.notna()]
y_train = y[y.notna()]

# Initialize the linear regression model and fit it to the data
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the 'Dauer_in_Sekunden' for missing values
X_missing = X[y.isna()]
print(X_missing)

In [ ]:
predicted_durations = model.predict(X_missing)
predicted_durations

In [ ]:

# Fill the missing 'Dauer_in_Sekunden' with the predicted values
df8.loc[y.isna(), 'Dauer_in_Sekunden'] = predicted_durations
df8.isna().sum()

In [ ]:
missing_indices = df8.loc[df8['End_Time'].isna()].index
# Impute the missing 'End_Time' values using the predicted durations
df8.loc[missing_indices, 'End_Time'] = df8.loc[missing_indices, 'Start'] + pd.to_timedelta(df8.loc[missing_indices, 'Dauer_in_Sekunden'], unit='s')
df8.isna().sum()

In [ ]:
# Clean up the DataFrame by dropping intermediate columns if desired
df8 = df8.drop(['Dauer', 'Dauer_in_Sekunden'], axis=1)

# View the updated DataFrame
df8.isna().sum()

In [ ]:
df9 = df8.copy()

In [ ]:
df9.dtypes

In [ ]:
df9.shape

In [ ]:
df9.isna().sum()

# Some final sentence cleaning

In [ ]:
# Splitting each string in 'your_column' at the pattern and keeping only the part before it
df9['Text_Spoken'] = df9['Text_Spoken'].apply(lambda x: x.split('Geschiedenen Drucksachen')[0] + '.' if 'Geschiedenen Drucksachen' in x else x)
df9['Text_Spoken'] = df9['Text_Spoken'].apply(lambda x: x.split('Drucksachen')[0] + '.' if 'Geschiedenen Drucksachen' in x else x)
df9['Text_Spoken'] = df9['Text_Spoken'].replace('\d{2}\/\d{4}\,', '', regex=True)
df9['Text_Spoken'] = df9['Text_Spoken'].replace('\d{2}\/\d{4}', '', regex=True)

In [ ]:
print('This worked')

In [ ]:
#df9.to_csv('/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Final_Project/speeches_bundestag/Final_DF/final_text_speaker_df_2.csv', index=False, encoding='utf-8')